In [1]:
cd ../..

D:\Users\mrefoyo\Proyectos\Sub-SpaCE_plus


In [2]:
import os
import pickle
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf

from experiments.evaluation.evaluation_utils import load_dataset_for_eval, calculate_metrics_for_dataset

print(tf.__version__)

2.13.0


In [18]:
# DATASETS = ['CBF', 'chinatown', 'coffee', 'gunpoint', 'ECG200']
DATASETS = ["BasicMotions", "NATOPS"]
MO_UTILITY = np.array([0.0, 0.2*1, 0.8*1, 0.0])
model_to_explain = "cls_basic_train"
ae_name = "ae_basic_train"
methods = {
    # Comparison between single objective and multi objective ordered by same weights
    "abd6be72d2e5c0967af2134fe463ff2ad9250524": "subspace_gknn_cch_figradcam",
    "7dbaa6cfdb5cd23ba93e6495b43a5f2874fd79c0": "multisubspace_gknn_cch_75_25",
    "b4a4daf2c1791e984fb1ec07dfe589ddf9caf4bb": "multisubspace_gknn_cch_55_100",
}



# Get results

In [19]:
# Data and aux data dict
data_dict = {}
models_dict = {}
outlier_calculators_dict = {}
possible_nuns_dict = {}
desired_classes_dict = {}
original_classes_dict = {}

# Results dicts
mean_results_dict = {}
methods_cfs_dict = {}
results_all_datasets_df = pd.DataFrame()
common_test_indexes_dict = {}

# Iterate through datasets
for dataset in DATASETS:
    print(f'Calculating metrics for {dataset}')
    # Load all info needed to get the counterfactual
    data_tuple, original_classes, model, outlier_calculator, possible_nuns, desired_classes = load_dataset_for_eval(dataset, model_to_explain, ae_name)
    data_dict[dataset] = data_tuple
    models_dict[dataset] = model
    outlier_calculators_dict[dataset] = outlier_calculator
    possible_nuns_dict[dataset] = possible_nuns
    desired_classes_dict[dataset] = desired_classes
    original_classes_dict[dataset] = original_classes

    # Get the metrics for all methods
    dataset_mean_std_df, dataset_results_df, method_cfs_dataset, common_test_indexes = calculate_metrics_for_dataset(
        dataset, methods, model_to_explain,
        data_tuple, original_classes, model, outlier_calculator, possible_nuns,
        mo_weights=MO_UTILITY
    )
    mean_results_dict[dataset] = dataset_mean_std_df
    methods_cfs_dict[dataset] = method_cfs_dataset
    results_all_datasets_df = pd.concat([results_all_datasets_df, dataset_results_df])
    common_test_indexes_dict[dataset] = common_test_indexes

    # Store results
    dataset_mean_std_df.to_csv(f'./experiments/evaluation/results_mean_metrics_{dataset}.csv', sep=";", index=False)

# Store all results
results_all_datasets_df.to_csv(f'./experiments/evaluation/results_all.csv', sep=";", index=False)


Calculating metrics for BasicMotions
7dbaa6cfdb5cd23ba93e6495b43a5f2874fd79c0


100%|██████████| 40/40 [00:43<00:00,  1.09s/it]


abd6be72d2e5c0967af2134fe463ff2ad9250524


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]


b4a4daf2c1791e984fb1ec07dfe589ddf9caf4bb


...

100%|██████████| 100/100 [01:58<00:00,  1.18s/it]


abd6be72d2e5c0967af2134fe463ff2ad9250524


100%|██████████| 100/100 [00:57<00:00,  1.75it/s]


b4a4daf2c1791e984fb1ec07dfe589ddf9caf4bb


100%|██████████| 100/100 [02:01<00:00,  1.21s/it]


In [20]:
from IPython.display import display
for dataset in DATASETS:
    print(dataset)
    display(mean_results_dict[dataset])

BasicMotions


,method,ii,nchanges,sparsity,L1,L2,proba,valid,nuns_valid,outlier_score,increase_outlier_score,subsequences,subsequences %,times
0,multisubspace_gknn_cch_75_25,19.5 ± 11.69,184.92 ± 113.18,0.31 ± 0.19,712.97 ± 779.03,64.01 ± 51.87,0.46 ± 0.09,1.0 ± 0.0,1.0 ± 0.0,0.3 ± 0.19,0.01 ± 0.02,10.15 ± 3.56,0.03 ± 0.01,27.13 ± 5.58
1,subspace_gknn_cch_figradcam,19.5 ± 11.69,279.55 ± 147.62,0.47 ± 0.25,1003.23 ± 992.97,75.73 ± 58.38,0.71 ± 0.14,1.0 ± 0.0,1.0 ± 0.0,0.23 ± 0.15,0.01 ± 0.02,9.15 ± 4.67,0.03 ± 0.02,24.82 ± 8.71
2,multisubspace_gknn_cch_55_100,19.5 ± 11.69,153.6 ± 96.66,0.26 ± 0.16,651.23 ± 682.21,63.13 ± 49.78,0.44 ± 0.08,1.0 ± 0.0,1.0 ± 0.0,0.32 ± 0.2,0.01 ± 0.01,8.82 ± 3.34,0.03 ± 0.01,42.16 ± 6.52


NATOPS


,method,ii,nchanges,sparsity,L1,L2,proba,valid,nuns_valid,outlier_score,increase_outlier_score,subsequences,subsequences %,times
0,multisubspace_gknn_cch_75_25,87.18 ± 53.72,250.25 ± 125.67,0.2 ± 0.1,111.93 ± 93.02,8.87 ± 5.63,0.64 ± 0.15,1.0 ± 0.0,1.0 ± 0.0,0.62 ± 0.21,0.08 ± 0.07,23.05 ± 3.49,0.04 ± 0.01,31.04 ± 4.07
1,subspace_gknn_cch_figradcam,87.18 ± 53.72,355.11 ± 145.58,0.29 ± 0.12,162.04 ± 122.03,10.72 ± 6.46,0.93 ± 0.07,1.0 ± 0.0,1.0 ± 0.0,0.57 ± 0.18,0.03 ± 0.04,23.84 ± 3.42,0.04 ± 0.01,18.74 ± 5.49
2,multisubspace_gknn_cch_55_100,87.18 ± 53.72,230.95 ± 111.41,0.19 ± 0.09,109.65 ± 96.03,8.76 ± 5.81,0.67 ± 0.14,1.0 ± 0.0,1.0 ± 0.0,0.65 ± 0.23,0.11 ± 0.09,20.52 ± 4.72,0.03 ± 0.01,49.9 ± 2.55


In [21]:
columns = ["nchanges", "L2", "valid", "increase_outlier_score", "subsequences", "times"]
for dataset in DATASETS:
    print(dataset)
    display(mean_results_dict[dataset].set_index("method").sort_index()[columns])

BasicMotions


,nchanges,L2,valid,increase_outlier_score,subsequences,times
method,,,,,,
multisubspace_gknn_cch_55_100,153.6 ± 96.66,63.13 ± 49.78,1.0 ± 0.0,0.01 ± 0.01,8.82 ± 3.34,42.16 ± 6.52
multisubspace_gknn_cch_75_25,184.92 ± 113.18,64.01 ± 51.87,1.0 ± 0.0,0.01 ± 0.02,10.15 ± 3.56,27.13 ± 5.58
subspace_gknn_cch_figradcam,279.55 ± 147.62,75.73 ± 58.38,1.0 ± 0.0,0.01 ± 0.02,9.15 ± 4.67,24.82 ± 8.71


NATOPS


,nchanges,L2,valid,increase_outlier_score,subsequences,times
method,,,,,,
multisubspace_gknn_cch_55_100,230.95 ± 111.41,8.76 ± 5.81,1.0 ± 0.0,0.11 ± 0.09,20.52 ± 4.72,49.9 ± 2.55
multisubspace_gknn_cch_75_25,250.25 ± 125.67,8.87 ± 5.63,1.0 ± 0.0,0.08 ± 0.07,23.05 ± 3.49,31.04 ± 4.07
subspace_gknn_cch_figradcam,355.11 ± 145.58,10.72 ± 6.46,1.0 ± 0.0,0.03 ± 0.04,23.84 ± 3.42,18.74 ± 5.49


# Counterfactual visualization

In [10]:
import math
def plot_comp_cfs(x_orig, possible_nuns, x_cfs_dict):
    
    # Calculate auxiliary variables
    method_names = list(x_cfs_dict.keys())
    x_cfs = list(x_cfs_dict.values())
    length = x_orig.shape[0]
    n_channels = x_orig.shape[1]

    # Define outer grid
    fig = plt.figure(figsize=(22, 16))
    grid_columns = 2
    grid_rows = math.ceil(len(x_cfs) / grid_columns)
    outer_grid = gridspec.GridSpec(grid_rows, grid_columns, wspace=0.1, hspace=0.3)

    # Iterate through outer grid plots
    for m, outer in enumerate(outer_grid):
        try:
            # Get change mask 
            x_cf = x_cfs[m]
            method_name = method_names[m]
            # Get nuns used by the method depending on the name
            if "gknn" in method_name:
                nun_example = possible_nuns["gknn"]
            elif "iknn" in method_name:
                nun_example = possible_nuns["iknn"]
            else:
                raise ValueError('Not detected NUN finding procedure in name. Method name must contain "gknn" or "iknn"')
            found_counterfactual_mask = (x_orig != x_cf.reshape(1, length, n_channels)).astype(int)
            diff_mask = np.diff(found_counterfactual_mask, prepend=0, append=0, axis=1)
        except IndexError:
            break
    
        # Create inner grid
        inner_grid = gridspec.GridSpecFromSubplotSpec(n_channels, 1, wspace=0.2, hspace=0.2, subplot_spec=outer)
        sub_channel_axs = []
        for i in range(n_channels):
            ax = plt.Subplot(fig, inner_grid[i])
            ax.plot(nun_example.reshape(1, length, n_channels)[:, :, i].flatten(), color='grey')
            ax.plot(x_cf.reshape(1, length, n_channels)[:, :, i].flatten(), color='red')
            ax.plot(x_orig.reshape(1, length, n_channels)[:, :, i].flatten(), color="#332288")
            # Set title if first channel
            if i == 0:
                ax.set_title(method_name)
            
            # Add fill to changes
            starts = np.clip( np.where(diff_mask[0, :, i].flatten() == 1)[0] - 1 , 0, length)
            ends = np.where(diff_mask[0, :, i].flatten() == -1)[0]
            # Iterate over the mask segments and fill them with red color
            for i in range(0, len(starts)):
                start_idx = starts[i]
                end_idx = ends[i]
                # plt.fill_between(comp_df.iloc[start_idx:end_idx].index, comp_df.min(), comp_df.max(), color='red', alpha=0.7)
                ax.axvspan(start_idx, end_idx, alpha=0.2, color='red')
            
            # Add ax to list of subplots
            fig.add_subplot(ax)
            sub_channel_axs.append(ax)

    # Plot figure
    fig.legend(
        sub_channel_axs,     # The line objects
        labels=['NUN', 'x_cf', 'x_orig'],   # The labels for each line
        loc='center',
        prop={'size': 12}
    )
    plt.show()



In [11]:
dataset = "UWaveGestureLibrary"
results_all_datasets_df[results_all_datasets_df["dataset"]==dataset]

,ii,nchanges,sparsity,L1,L2,proba,valid,nuns_valid,outlier_score,increase_outlier_score,subsequences,subsequences %,times,method,order,dataset
0,3,630.0,0.666667,849.672028,40.468249,0.994502,True,NaN,0.357573,0.000000,1.0,0.002116,1291.526145,comte,1,UWaveGestureLibrary
1,13,945.0,1.000000,701.469408,29.498525,0.986486,True,NaN,0.394286,0.000000,1.0,0.002116,1301.269161,comte,1,UWaveGestureLibrary
2,14,630.0,0.666667,534.941475,27.801814,0.981622,True,NaN,0.429190,0.000000,1.0,0.002116,1301.799412,comte,1,UWaveGestureLibrary
3,15,945.0,1.000000,753.060521,31.011630,0.989560,True,NaN,0.375455,0.000000,1.0,0.002116,1309.426037,comte,1,UWaveGestureLibrary
4,19,630.0,0.666667,546.554781,26.437987,0.989558,True,NaN,0.478830,0.000000,1.0,0.002116,1316.859315,comte,1,UWaveGestureLibrary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,300,158.0,0.167196,144.647351,12.668673,0.772654,True,1.0,0.487605,0.041953,3.0,0.006349,48.047095,subspace_gknn_cch_mut07,2,UWaveGestureLibrary
96,302,84.0,0.088889,82.245441,9.598369,0.449715,True,1.0,0.591355,0.123180,5.0,0.010582,44.687890,subspace_gknn_cch_mut07,2,UWaveGestureLibrary
97,303,43.0,0.045503,38.132580,6.282867,0.620453,True,1.0,0.525460,0.036501,2.0,0.004233,43.703472,subspace_gknn_cch_mut07,2,UWaveGestureLibrary
98,314,570.0,0.603175,556.326838,27.585234,0.814680,True,1.0,0.667473,0.013069,6.0,0.012698,83.094417,subspace_gknn_cch_mut07,2,UWaveGestureLibrary


In [12]:
# methods = ["subspace_grouped", "subspace_v2_grouped", "subspace_individual", "subspace_v2_individual"]
common_index_instance = 13

In [13]:
# Get data
X_train, y_train, X_test, y_test = data_dict[dataset]
common_indexes = common_test_indexes_dict[dataset]
instance_index = common_indexes[common_index_instance]

# Get common cfs calculated for all method
x_orig = X_test[instance_index]

# Get possible NUNs
possible_nuns = possible_nuns_dict[dataset]
possible_nuns_instance = {type: possible_nuns[type][instance_index] for type in ["gknn", "iknn"]}

# Get original class and desired class
original_class = original_classes_dict[dataset][instance_index]
desired_class = desired_classes_dict[dataset][instance_index]

# Get the cf calculated for all methods
sol_cfs_methods = methods_cfs_dict[dataset]
sol_cfs_methods = {method: sol_cfs[common_index_instance]['cf'] for method, sol_cfs in sol_cfs_methods.items()}

# Show result
print(f'Sample {instance_index}: true class {y_test[instance_index]}, x_orig predicted {original_class}, desired_class  {desired_class}')
plot_comp_cfs(x_orig, possible_nuns_instance, sol_cfs_methods)


Sample 51: true class 1, x_orig predicted 1, desired_class  6


ValueError: Not detected NUN finding procedure in name. Method name must contain "gknn" or "iknn"

<Figure size 2200x1600 with 0 Axes>

In [ ]:
# Create plot with examples of each method in each dataset

In [ ]:
# CBF: 1, 2, 10, 18, 19
# chinatown: 3, 7, 19
# coffee: 1, 3, 9, 16
# ECG200: 0, 13
# gunpoint: 9, 10, 15 (pesimos en general)
dataset_samples = [('CBF', 2), ('chinatown', 19), ('coffee', 3), ('ECG200', 13), ('gunpoint', 15)]
methods_trad_dict = {"wcf_ng": "W-CF", "ng": "NG", "ab_cf": "AB-CF", "subspace": "Sub-SpaCE"}
methods = list(methods_trad_dict.keys())
include_nun_list = [False, False, False, False]

In [ ]:
from collections import OrderedDict

# Define outer grid
fig = plt.figure(figsize=(16, 6))
grid_rows = len(methods)
grid_columns = len(dataset_samples)
outer_grid = gridspec.GridSpec(grid_rows, grid_columns, wspace=0, hspace=0)

# Iterate through outer grid plots
for i_outer, outer in enumerate(outer_grid):
    i_dataset = i_outer % grid_columns
    dataset, instance = dataset_samples[i_dataset]
    column_title = f'{dataset} instance {instance}'
    
    X_train, y_train, X_test, y_test = data_dict[dataset]
    nuns_idx = nuns_idx_dict[dataset]
    sol_cfs_methods = sol_cfs_dict[dataset]
    sol_cfs_methods = {method: sol_cfs_methods[method][instance]['cf'] for method in methods}

    # Calculate auxiliary variables
    method_names = list(sol_cfs_methods.keys())
    x_cfs = list(sol_cfs_methods.values())
    x_orig = X_test[instance]
    nun_example = X_train[nuns_idx[instance]]
    length = x_orig.shape[0]
    n_channels = x_orig.shape[1]

    # get method m
    m = i_outer // grid_columns
    try:
        # Get change mask 
        x_cf = x_cfs[m]
        method_name = method_names[m]
        found_counterfactual_mask = (x_orig != x_cf.reshape(1, length, n_channels)).astype(int)
        diff_mask = np.diff(found_counterfactual_mask, prepend=0, append=0, axis=1)
    except IndexError:
        break

    # Create inner grid
    inner_grid = gridspec.GridSpecFromSubplotSpec(n_channels, 1, wspace=0, hspace=0, subplot_spec=outer)
    sub_channel_axs = []
    for i in range(n_channels):
        ax = plt.Subplot(fig, inner_grid[i])
        if include_nun_list[m]:
            ax.plot(nun_example.reshape(1, length, n_channels)[:, :, i].flatten(), color='grey')
        ax.plot(x_cf.reshape(1, length, n_channels)[:, :, i].flatten(), color='red')
        ax.plot(x_orig.reshape(1, length, n_channels)[:, :, i].flatten(), color="#332288")
        # ax.axis("off")
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Add fill to changes
        starts = np.clip( np.where(diff_mask[0, :, i].flatten() == 1)[0] - 1 , 0, length)
        ends = np.where(diff_mask[0, :, i].flatten() == -1)[0]
        # Iterate over the mask segments and fill them with red color
        for i in range(0, len(starts)):
            start_idx = starts[i]
            end_idx = ends[i]
            # plt.fill_between(comp_df.iloc[start_idx:end_idx].index, comp_df.min(), comp_df.max(), color='red', alpha=0.7)
            ax.axvspan(start_idx, end_idx, alpha=0.2, color='red')
        
        # Set column titles
        if m == 0:
            ax.set_title(column_title)
        if i_dataset == 0:
            ax.set_ylabel(methods_trad_dict[method_name], size='large')
        # Add ax to list of subplots
        fig.add_subplot(ax)
        sub_channel_axs.append(ax)

# Plot figure
plt.tight_layout()
"""fig.legend(
    sub_channel_axs,     # The line objects
    labels=['$x^n$', 'x', "x'"],   # The labels for each line
    loc='lower center',
    # loc='center right',
    prop={'size': 14},
    ncol=3
)"""
# plt.show()
# plt.savefig('./experiments/results/examples_cfs.png', dpi=300)


In [ ]:
from collections import OrderedDict

# Define outer grid
fig = plt.figure(figsize=(12, 12))
grid_rows = len(dataset_samples)
grid_columns = len(methods)
outer_grid = gridspec.GridSpec(grid_rows, grid_columns, wspace=0, hspace=0)

# Iterate through outer grid plots
for i_outer, outer in enumerate(outer_grid):
    i_dataset = i_outer // grid_columns
    dataset, instance = dataset_samples[i_dataset]
    
    X_train, y_train, X_test, y_test = data_dict[dataset]
    nuns_idx = nuns_idx_dict[dataset]
    sol_cfs_methods = sol_cfs_dict[dataset]
    sol_cfs_methods = {method: sol_cfs_methods[method][instance]['cf'] for method in methods}

    # Calculate auxiliary variables
    method_names = list(sol_cfs_methods.keys())
    x_cfs = list(sol_cfs_methods.values())
    x_orig = X_test[instance]
    nun_example = X_train[nuns_idx[instance]]
    length = x_orig.shape[0]
    n_channels = x_orig.shape[1]

    # get method m
    m = i_outer % grid_columns
    try:
        # Get change mask 
        x_cf = x_cfs[m]
        method_name = method_names[m]
        found_counterfactual_mask = (x_orig != x_cf.reshape(1, length, n_channels)).astype(int)
        diff_mask = np.diff(found_counterfactual_mask, prepend=0, append=0, axis=1)
    except IndexError:
        break
    
    # Create inner grid
    inner_grid = gridspec.GridSpecFromSubplotSpec(n_channels, 1, wspace=0, hspace=0, subplot_spec=outer)
    sub_channel_axs = []
    for i in range(n_channels):
        ax = plt.Subplot(fig, inner_grid[i])
        if include_nun_list[m]:
            ax.plot(nun_example.reshape(1, length, n_channels)[:, :, i].flatten(), color='grey')
        ax.plot(x_cf.reshape(1, length, n_channels)[:, :, i].flatten(), color='red')
        ax.plot(x_orig.reshape(1, length, n_channels)[:, :, i].flatten(), color="#332288")
        # ax.axis("off")
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Add fill to changes
        starts = np.clip( np.where(diff_mask[0, :, i].flatten() == 1)[0] - 1 , 0, length)
        ends = np.where(diff_mask[0, :, i].flatten() == -1)[0]
        # Iterate over the mask segments and fill them with red color
        for i in range(0, len(starts)):
            start_idx = starts[i]
            end_idx = ends[i]
            # plt.fill_between(comp_df.iloc[start_idx:end_idx].index, comp_df.min(), comp_df.max(), color='red', alpha=0.7)
            ax.axvspan(start_idx, end_idx, alpha=0.2, color='red')
        
        # Set column titles
        column_title = methods_trad_dict[method_name]
        # row_title = f'{dataset} \n instance {instance}'
        row_title = f'{dataset} [{instance}]'
        if m == 0:
            ax.set_ylabel(row_title, fontsize=16, rotation=90)
        if i_dataset == 0:
            ax.set_title(column_title, fontsize=18)
            
        # Add ax to list of subplots
        fig.add_subplot(ax)
        sub_channel_axs.append(ax)

# Plot figure
plt.tight_layout()
"""fig.legend(
    sub_channel_axs,     # The line objects
    labels=['$x^n$', 'x', "x'"],   # The labels for each line
    # loc='lower center',
    loc='center right',
    prop={'size': 18},
    ncol=1
)"""
# plt.show()
plt.savefig('./experiments/results/examples_cfs.png', dpi=300)
